# Simple Linear Regression #

### Treasury Bonds Holdins vs US Dollar Index ###

In [1]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Statistics
import statsmodels.api as sm 
import matplotlib.pyplot as plt

# Get Data
from fredapi import Fred

# Pretty Notation
from IPython.display import display, Math

In [2]:
# Data Collection Function from FRED
def get_fred_data(symbol: str) -> pd.DataFrame:
    
    fred_key = '0174cb93931388a2bf305663e4117fd3'

    fred = Fred(api_key = fred_key)
    
    df = fred.get_series(symbol)
    
    return df

In [3]:
# Data for Y (Memorandum Items: Custody Holdings: Marketable U.S. Treasury Securities: Wednesday Level)
y = get_fred_data('WMTSECL1')

# Creating Time Index
y.index = pd.to_datetime(y.index)

print(y)

In [4]:
# Data for Explanatory Variables (Nominal Broad U.S. Dollar Index)
x = get_fred_data('DTWEXBGS')

# Time Index
x.index = pd.to_datetime(x.index)

print(x)

In [33]:
# Create the Data for Lineal Regression
ols_df = pd.DataFrame({'x': x, 'y': y})

# Eliminate 0s
ols_df = ols_df[(ols_df['x'] > 0) & (ols_df['y'] > 0)]

# Cut the Sample
ols_df = ols_df["2018-01-01":]

ols_df

In [34]:
# We use Natural Log to calculate better betas
log_ols_df = np.log(ols_df)

# Drop NaNs
log_ols_df = log_ols_df.dropna()

# Set Index as a Date Item
log_ols_df.index = pd.to_datetime(log_ols_df.index)

log_ols_df

In [42]:
"""
By Plotting the Variables we might understand the relations among them
"""

# Create Figure
fig, ax1 = plt.subplots(dpi = 300)

# Holdings of Treasury Bonds Plot
log_ols_df['y'].plot(color = 'blue', ax = ax1)
ax1.set_xlabel('Date')
ax1.set_ylabel(
    'Holdings of Treasury Bonds', 
    color='blue'
    )

# US Dollar Index Plot
ax2 = ax1.twinx()

log_ols_df['x'].plot(color = 'red', ax = ax2)
ax2.set_ylabel(
    'US Dollar Index', 
    color='red'
    )

plt.show()

In [43]:
# Correlation Coefficient

display(Math(r"\rho=\frac{Cov(x,y)}{\sigma_x\sigma_y}"))

correlation = log_ols_df['y'].corr(log_ols_df['x'])

print(f"\nPearson Correlation Coefficient: {correlation}")

In [44]:
fig, ax1 = plt.subplots(dpi = 600)

plt.scatter(log_ols_df['x'], log_ols_df['y'])
plt.ylabel('Custody Holdings Rate')
plt.xlabel('Nominal Broad U.S. Dollar Index')

plt.show()

In [45]:
# Create a Box Plot to identify Outliers

plt.boxplot(log_ols_df['y'], vert=True, patch_artist=True)  

# Config
plt.title("Custody Holdings: Marketable U.S. Treasury Securities Boxplot")
plt.ylabel("Values")
plt.xticks([1], ["Data"]) 

plt.show()

In [46]:
# Create a Box Plot to identify Outliers

plt.boxplot(log_ols_df['x'], vert=True, patch_artist=True)  

# Config
plt.title("Nominal Broad U.S. Dollar Index Boxplot")
plt.ylabel("Values")
plt.xticks([1], ["Data"]) 

plt.show()

In [47]:
# Let us eliminate those values on the Unemployment Rate Growth Rate that exceeds 3 Standard Deviations

inferior_limit = log_ols_df['y'].mean() - 3 * log_ols_df['y'].std()
superior_limit = log_ols_df['y'].mean() + 3 * log_ols_df['y'].std()

# Filter outliers
ols_df_filtered = log_ols_df[(log_ols_df["y"] >= inferior_limit) & (log_ols_df["y"] <= superior_limit)]

ols_df_filtered

In [48]:
"""
In a Linear Regression Model it is fundamental to have a Constant Value
This can be expressed as a Vertical Vector of 1's'
"""

ols_df_filtered = sm.add_constant(ols_df_filtered)

ols_df_filtered

In [14]:
# The Simple Linear Regression Model Equation

display(Math(r"y=\beta_0+\beta_1x+\varepsilon"))

In [15]:
display(Math(r"\hat{\beta_0}=\bar{y}-\hat{\beta_1}\bar{x}"))

display(Math(r"\hat{\beta_1}=\frac{\sum{xy}-\bar{x}\bar{y}}{\sum{x^2}-\bar{x}^2}=\frac{Cov(x,y)}{Var(x)}"))

In [50]:
# Obtaining Variance and Covariance

covariance = ols_df_filtered['y'].cov(ols_df_filtered['x'])
variance = ols_df_filtered['x'].var()

print(f"Covariance: {covariance}")
print(f"Variance: {variance}")

In [51]:
# Obtaining Slope Coefficient
slope = covariance/variance

print(f"The beta coefficient is: {slope}")

In [52]:
# Obtaining Intercept Coefficient

intercept = ols_df_filtered['y'].mean() - slope*ols_df_filtered['x'].mean()

print(f"The alpha coefficient is: {intercept}")

In [53]:
# Obtaining Fitted Values and Residuals

ols_df_filtered['y_hat'] = intercept + slope*ols_df_filtered['x']
ols_df_filtered['residuals'] = ols_df_filtered['y'] - ols_df_filtered['y_hat']

ols_df_filtered

In [54]:
# Create Comparative Plot
plt.figure(figsize=(10, 6))
plt.plot(ols_df_filtered['y'], label='Real Holdings Values', color='blue', alpha=0.7)
plt.plot(ols_df_filtered['y_hat'], label='Fitted Holdings Values', color='green', alpha=0.7)

# Config
plt.title('Time Series')
plt.xlabel('Time')
plt.ylabel('Treasury Bonds Holdings')
plt.legend()

# Show
plt.show()

In [55]:
# Plot Residuals
plt.figure(figsize=(10, 6))
plt.plot(ols_df_filtered['residuals'], label='Regression Residuals', color='red', alpha=0.7)

# Config
plt.title('Time Series')
plt.xlabel('Time Index')
plt.ylabel('Residuals')
plt.legend()

# Show
plt.show()

In [56]:
# The Residuals Mean can give us information about the bias of the coefficients

print(f"Residuals Mean: {ols_df_filtered['residuals'].mean().round(3)}")

In [58]:
# Obtain R-Squared

display(Math(r"R^2=\frac{Var(\hat{y})}{Var(y)}"))

print("\n")
print(f"Fitted Values Variance: {ols_df_filtered['y_hat'].var()}")
print(f"Real Values Variance: {ols_df_filtered['y'].var()}")

In [59]:
# Then the R-Squared Coefficient

r2 = ols_df_filtered['y_hat'].var()/ols_df_filtered['y'].var()

print(f"The R-Squared Coefficient is: {r2}")

In [60]:
# The Betas Standard Errors

display(Math(r"Var(\hat{\beta_0})=\sigma^2(\frac{1}{n}+\frac{\bar{x}^2}{(n-1)Var(x)})"))
display(Math(r"Var(\hat{\beta_1})=\frac{\sigma^2}{nVar(x)}"))

In [61]:
# Let us calculate the Slope Standard Deviation

residuals_var = ols_df_filtered['residuals'].var(ddof=1)
x_var = ols_df_filtered['x'].var()
n = len(ols_df_filtered)

print(f"Residuals Variance: {residuals_var}")
print(f"Independent Variable Variance: {x_var}")

In [62]:
slope_std = np.sqrt(residuals_var)/np.sqrt(n*x_var)

print(f"Slope Coefficient Standard Deviation: {slope_std}")

In [63]:
intercept_var = residuals_var * (1/n + ols_df_filtered['x'].mean()**2 / ((n - 1) * x_var))
intercept_std = np.sqrt(intercept_var)

print(f"Intercept Coefficient Standard Deviation: {intercept_std}")

In [64]:
# Create the t values

display(Math(r"t=\frac{\beta}{sse(\beta)}"))

In [65]:
# Calculate the t values

slope_t = slope/slope_std
intercept_t = intercept/intercept_std

if abs(slope_t) > 2:
    print(f"The t-value of {slope_t} shows evidence of statistical significance for the slope coefficient")
else:
    print(f"The t-value of {slope_t} shows no evidence of statistical significance for the slope coefficient")

if abs(intercept_t) > 2:
    print(f"The t-value of {intercept_t} shows evidence of statistical significance for the intercept coefficient")
else:
    print(f"The t-value of {intercept_t} shows no evidence of statistical significance for the intercept coefficient")

In [66]:
#Model specification
model = sm.OLS(
    ols_df_filtered['y'], 
    sm.add_constant(ols_df_filtered['x'])
    )   
     
#the results of the model
results = model.fit() 
    
#The Parameters
Beta2 = results.params  

#here we check the summary
print(results.summary())       
